In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

Matplotlib is building the font cache; this may take a moment.


In [2]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Data/"

In [3]:
pd.set_option('display.max_colwidth', None)
summary = pd.read_adls(path + "batch_mQA10T_i2112084_d20230323043130_creditinquiry_LN_stats", reader = pd.read_csv, sep = "\t")
summary["month"] = summary["file_name"].str.split("_").str[-1]
summary = summary.append(summary.sum(numeric_only=True), ignore_index=True)
summary

/tmp/ipykernel_27019/895700462.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(summary.sum(numeric_only=True), ignore_index=True)


,file_name,file_date_time,number_of_records_processed,number_of_unique_records,number_of_records_rejected_with_search_information_missing,number_of_duplicate_records,number_of_records_with_sbfe_contribution_id_missing,number_of_records_with_date_time_missing,number_of_records_with_platform_type_missing,number_of_records_with_hit_nohit_indicator_missing,number_of_records_containing_toomanydelimiters,number_of_skewed_records,month
0,batch_mCVLNB_i2095718_d20221010184211_LNRS_INQUIRY_202201,20221010184211,197432,197432,12873,0,197432,0,0,197432,0,0,202201
1,batch_mCVLNB_i2095722_d20221010184226_LNRS_INQUIRY_202109,20221010184226,2180106,2180106,10799,0,2180106,0,0,2180106,0,0,202109
2,batch_mCVLNB_i2095736_d20221010184333_LNRS_INQUIRY_202111,20221010184333,101486,101486,11389,0,101486,0,0,101486,0,0,202111
3,batch_mCVLNB_i2095730_d20221010184302_LNRS_INQUIRY_202202,20221010184302,107671,107671,18353,0,107671,0,0,107671,0,0,202202
4,batch_mCVLNB_i2095732_d20221010184309_LNRS_INQUIRY_202108,20221010184309,100148,100148,10397,0,100148,0,0,100148,0,0,202108
5,batch_mCVLNB_i2095716_d20221010184202_LNRS_INQUIRY_202112,20221010184202,2241685,2241685,24604,0,2241685,0,0,2241685,0,0,202112
6,batch_mCVLNB_i2095724_d20221010184235_LNRS_INQUIRY_202107,20221010184235,345187,345187,10174,0,345187,0,0,345187,0,0,202107
7,batch_mCVLNB_i2095720_d20221010184218_LNRS_INQUIRY_202110,20221010184218,324115,324115,11712,0,324115,0,0,324115,0,0,202110
8,batch_mCVLNB_i2095726_d20221010184243_LNRS_INQUIRY_202204,20221010184243,461617,461617,13180,0,461617,0,0,461617,0,0,202204
9,batch_mCVLNB_i2095728_d20221010184252_LNRS_INQUIRY_202206,20221010184252,2769276,2481590,27455,287686,2769276,0,0,2769276,0,0,202206


In [4]:
freq(summary["month"])

,Count,Pct,Cuml Count,Cuml Pct
month,,,,
202107,1.0,0.076923,1.0,0.076923
202108,1.0,0.076923,2.0,0.153846
202109,1.0,0.076923,3.0,0.230769
202110,1.0,0.076923,4.0,0.307692
202111,1.0,0.076923,5.0,0.384615
202112,1.0,0.076923,6.0,0.461538
202201,1.0,0.076923,7.0,0.538462
202202,1.0,0.076923,8.0,0.615385
202203,1.0,0.076923,9.0,0.692308


## Import Actual tradeline

In [11]:
path_ln_reject = "batch_mQA10T_i2112088_d20230323043922_creditinquiry_LN_rejected_records_file"
path_part1 = "batch_mQA10T_i2112100_d20230323044334_creditinquiry_external_file_part1of9"
path_part2 = "batch_mQA10T_i2112108_d20230323044639_creditinquiry_external_file_part2of9"
path_part3 = "batch_mQA10T_i2112102_d20230323044436_creditinquiry_external_file_part3of9"
path_part4 = "batch_mQA10T_i2112104_d20230323044534_creditinquiry_external_file_part4of9"
path_part5 = "batch_mQA10T_i2112090_d20230323044011_creditinquiry_external_file_part5of9"
path_part6 = "batch_mQA10T_i2112094_d20230323044146_creditinquiry_external_file_part6of9"
path_part7 = "batch_mQA10T_i2112092_d20230323044100_creditinquiry_external_file_part7of9"
path_part8 = "batch_mQA10T_i2112096_d20230323044233_creditinquiry_external_file_part8of9"
path_part9 = "batch_mQA10T_i2112098_d20230323044327_creditinquiry_external_file_part9of9"

In [12]:
names = ["SBFE Credit Inquiry ID", "Record Type", "Version Number", "Date/Time", "Business Name", "DBA", "Address1", "Address2", "City", "State", "Zip4", 
         "Contry Code", "Phone Number", "SSN"]
specs = [(0, 18), (18, 21), (21, 25), (25, 48), (48, 138), (138, 228), (228, 292), (292, 332), (332, 362), (362, 364), (364, 374), (374, 376), 
         (376, 390), (390, 401)]

In [13]:
part1 = pd.read_adls(path + path_part1, reader = pd.read_fwf, dtype = str, header = None, names = names, colspecs = specs, encoding = 'iso-8859-1')
part2 = pd.read_adls(path + path_part2, reader = pd.read_fwf, dtype = str, header = None, names = names, colspecs = specs, encoding = 'iso-8859-1')
part3 = pd.read_adls(path + path_part3, reader = pd.read_fwf, dtype = str, header = None, names = names, colspecs = specs, encoding = 'iso-8859-1')
part4 = pd.read_adls(path + path_part4, reader = pd.read_fwf, dtype = str, header = None, names = names, colspecs = specs, encoding = 'iso-8859-1')
part5 = pd.read_adls(path + path_part5, reader = pd.read_fwf, dtype = str, header = None, names = names, colspecs = specs, encoding = 'iso-8859-1')
part6 = pd.read_adls(path + path_part6, reader = pd.read_fwf, dtype = str, header = None, names = names, colspecs = specs, encoding = 'iso-8859-1')
part7 = pd.read_adls(path + path_part7, reader = pd.read_fwf, dtype = str, header = None, names = names, colspecs = specs, encoding = 'iso-8859-1')
part8 = pd.read_adls(path + path_part8, reader = pd.read_fwf, dtype = str, header = None, names = names, colspecs = specs, encoding = 'iso-8859-1')
part9 = pd.read_adls(path + path_part9, reader = pd.read_fwf, dtype = str, header = None, names = names, colspecs = specs, encoding = 'iso-8859-1')

In [14]:
print(part1.shape)
print(part2.shape)
print(part3.shape)
print(part4.shape)
print(part5.shape)
print(part6.shape)
print(part7.shape)
print(part8.shape)
print(part9.shape)
# expect 11,475,727
part1.shape[0] + part2.shape[0] + part3.shape[0] + part4.shape[0] + part5.shape[0] + part6.shape[0] + part7.shape[0] + part8.shape[0] + part9.shape[0]

(13250000, 14)
(13250000, 14)
(13250000, 14)
(13250000, 14)
(13250000, 14)
(13250000, 14)
(13250000, 14)
(13250000, 14)
(154116, 14)


106154116

In [15]:
part1.to_adls(path + "part1.parquet", format = '.parquet', overwrite = True)
part2.to_adls(path + "part2.parquet", format = '.parquet', overwrite = True)
part3.to_adls(path + "part3.parquet", format = '.parquet', overwrite = True)
part4.to_adls(path + "part4.parquet", format = '.parquet', overwrite = True)
part5.to_adls(path + "part5.parquet", format = '.parquet', overwrite = True)
part6.to_adls(path + "part6.parquet", format = '.parquet', overwrite = True)
part7.to_adls(path + "part7.parquet", format = '.parquet', overwrite = True)
part8.to_adls(path + "part8.parquet", format = '.parquet', overwrite = True)
part9.to_adls(path + "part9.parquet", format = '.parquet', overwrite = True)

In [16]:
stacked = pd.concat([part1, part2, part3, part4, part5, part6, part7, part8, part9], ignore_index = True)
print(stacked.shape)
del [part1, part2, part3, part4, part5, part6, part7, part8, part9]
stacked.to_adls(path + "stacked9parts.parquet", format = '.parquet', overwrite = True)

(106154116, 14)


In [17]:
stacked = stacked.fillna("")
stacked.to_adls(path + "stacked9parts.parquet", format = '.parquet', overwrite = True)

In [6]:
stacked[stacked["SBFE Credit Inquiry ID"] == "SBFECreditInqID"]

,SBFE Credit Inquiry ID,Record Type,Version Number,Date/Time,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN
0,SBFECreditInqID,Typ,Ver,Date_Time,Business_Name,Doing_Business_As,Address1,Address2,City,St,ZIP_Code,CC,Phone_Number,TaxID_SSN


## Analysis

In [3]:
stacked = pd.read_adls(path + "stacked9parts.parquet", reader = pd.read_parquet)

In [8]:
stacked = stacked[stacked["SBFE Credit Inquiry ID"] != "SBFECreditInqID"]
stacked.shape

(106154115, 14)

In [4]:
stacked.shape

(106154116, 14)

In [7]:
stacked.shape

(106154107, 14)

In [4]:
stacked.columns

Index(['SBFE Credit Inquiry ID', 'Record Type', 'Version Number', 'Date/Time',
       'Business Name', 'DBA', 'Address1', 'Address2', 'City', 'State', 'Zip4',
       'Contry Code', 'Phone Number', 'SSN'],
      dtype='object')

In [6]:
freq(stacked["Record Type"])

,Count,Pct,Cuml Count,Cuml Pct
Record Type,,,,
CI,106154107.0,1.0,106154107.0,1.0


In [7]:
freq(stacked["Version Number"])

,Count,Pct,Cuml Count,Cuml Pct
Version Number,,,,
0.1,106154107.0,1.0,106154107.0,1.0


In [9]:
temp = stacked[["SBFE Credit Inquiry ID", 'Date/Time']]
temp.columns = ["AccountNumber", "HistoryDateTime"]
temp.to_adls(path + "temp_stacked_datetime_for_merge.parquet", format = '.parquet', overwrite = True)

Bad pipe message: %s [b'^\x11d\x1b\xd2\x80xj\x8a*q2\x19\x94\xd2\xab%\xd4 \xe2ZT\xf3\x07]\x02\xf4\x14\xbb\x8f\xb7\x84\x19\xb8w\xc6\xa1\xc6"\x1c\xc2\xe7\x1f@an\xf1\xa6\xd4\xb2\xa1\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06']
Bad pipe message: %s [b'\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x83\xb5\xf16\xa8\x99\xf5\x07\xdd4\x00\n\x19\xd5)\xd7\xe1\xe3\xbc&\x13\x13']
Bad pipe message: %s [b"\x19\x1a\xa4T#\xa4x\xff`d\xa7\x1f\x86\x0c\xcd\xf9\xf5\xd5\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0

In [6]:
# stacked["Date/Time"]
stacked["date"] = stacked["Date/Time"].str.slice(0,7)
freq(stacked["date"])

,Count,Pct,Cuml Count,Cuml Pct
date,,,,
2019-11,18375.0,1.730974e-04,18375.0,0.000173
2021-05,322685.0,3.039779e-03,341060.0,0.003213
2021-06,7970157.0,7.508100e-02,8311217.0,0.078294
2021-07,10841996.0,1.021345e-01,19153213.0,0.180428
2021-08,5004185.0,4.714076e-02,24157398.0,0.227569
2021-09,9669571.0,9.108994e-02,33826969.0,0.318659
2021-10,8979584.0,8.459008e-02,42806553.0,0.403249
2021-11,7107134.0,6.695110e-02,49913687.0,0.470200
2021-12,10032822.0,9.451186e-02,59946509.0,0.564712


In [16]:
from dsgtools.reporting import TableWriter
wb = TableWriter(filename = "./_temp/test.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
wb.write_table(
    freq(stacked["State"]),
    sheetname="state",
)
wb.close()

In [23]:
stacked[stacked["State"] == "A8"]

,SBFE Credit Inquiry ID,Record Type,Version Number,Date/Time,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN
366682,AAA000000000762498,CI,0.1,2021-06-02T08:15:09,CAMBRIDGE MIDDLE EAST INTERIORS L.L.C,None,"Dubai Mall Office No 1201, 12th Floor, Emaar",None,Dubai,A8,None,US,44484622,None
2585867,AAA000000005443765,CI,0.1,2021-06-08T10:55:08,ELECTROMECHANICAL COMPANY LLC,None,"Al Ghaith Tower 702, Hamdan Street",None,Abu Dhabi,A8,None,US,26272164,None
2585889,AAA000000005443788,CI,0.1,2021-06-08T10:55:59,MINE MOBILE PHONES TRADING,None,15724,None,Dubai,A8,None,US,None,None
2829364,AAA000000005702849,CI,0.1,2021-06-14T11:16:39,Al Ahli Genral Treding Company,None,P.O. Box 206,None,Fujairah,A8,None,US,92226663,None
2830359,AAA000000005703877,CI,0.1,2021-06-14T11:59:20,Transgulf Aluminum Industry Company (L.L.C),None,P.O. Box 55409,None,Dubai,A8,None,US,48801524,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86509837,AAA000000099346509,CI,0.1,2022-05-17T08:54:20,AL TAIF CO GENERAL,None,None,None,Dubai,A8,None,US,24465577,None
86554455,AAA000000099393266,CI,0.1,2022-05-19T02:15:18,T T F T SECURITY SYSTEMS & EQUIPMENT TRADING L.L.C,None,"Sheilkh Zayed Road Office 54, Building 13, Al Quoz",None,Dubai,A8,None,US,42954764,None
87662084,AAA000000100745780,CI,0.1,2022-05-24T13:48:31,BALAJI TEXTILE CO. (L.L.C),None,64825,None,Dubai,A8,None,US,None,None
87718306,AAA000000100804621,CI,0.1,2022-05-27T01:45:06,JASHANMAL NATIONAL CO. LLC,None,"Behind Mall of Emirates Jashanmal Building, Al Bar",None,Dubai,A8,None,US,44174800,None


In [15]:
stacked.isnull().sum()

SBFE Credit Inquiry ID            0
Record Type                       0
Version Number                    0
Date/Time                         7
Business Name                  8743
DBA                       106134484
Address1                    4921120
Address2                   95523410
City                        9064085
State                       9030837
Zip4                       10197481
Contry Code                15233039
Phone Number               35086128
SSN                        92644640
dtype: int64

In [4]:
stacked["count"] = 1

#### round 1

In [7]:
stacked_roll_1 =stacked.groupby(by = ['Date/Time',
       'Business Name', 'DBA', 'Address1', 'Address2', 'City', 'State', 'Zip4',
       'Contry Code', 'Phone Number', 'SSN'])['count'].sum().reset_index(name='count')
freq("count", df = stacked_roll_1)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,97578864.0,9.585946e-01,97578864.0,0.958595
2,4128208.0,4.055466e-02,101707072.0,0.999149
3,61694.0,6.060691e-04,101768766.0,0.999755
4,16645.0,1.635170e-04,101785411.0,0.999919
5,4450.0,4.371588e-05,101789861.0,0.999963
...,...,...,...,...
103,1.0,9.823793e-09,101793673.0,1.000000
106,1.0,9.823793e-09,101793674.0,1.000000
117,1.0,9.823793e-09,101793675.0,1.000000


In [10]:
stacked_roll_1[stacked_roll_1["count"] == 10197]

,Date/Time,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN,count
19071975,2021-08-01T00:00:00,#,,#,,,,,,,,10197


In [17]:
temp = stacked[stacked["Business Name"] == "#"].copy()
print(temp.shape)
freq(temp.Address1)

(10198, 15)


,Count,Pct,Cuml Count,Cuml Pct
Address1,,,,
#,10197,1,10197,1
2202 TIIU CT,1,0,10198,1


In [20]:
stacked_roll_1.shape

(101793677, 12)

In [22]:
stacked_roll_1 = stacked_roll_1[stacked_roll_1["count"] < 10197]
fmt = make_format(cuts = [-np.inf, -1, 0, 1, 2, 3, 4, 5, 10, 50, 100, np.inf])
freq("count", df = stacked_roll_1, format = fmt, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,97578864,1,97578864,1
2,4128208,0,101707072,1
3,61694,0,101768766,1
4,16645,0,101785411,1
5,4450,0,101789861,1
6-10,3273,0,101793134,1
11-50,504,0,101793638,1
51-100,33,0,101793671,1
101+,5,0,101793676,1


#### round 2

In [5]:
stacked = stacked[stacked["Business Name"] != "#"]
print(stacked.shape)

(106143909, 15)


In [7]:
stacked["date"] = stacked["Date/Time"].str.slice(0,10)
stacked_roll_1 =stacked.groupby(by = ['date',
       'Business Name', 'DBA', 'Address1', 'Address2', 'City', 'State', 'Zip4',
       'Contry Code', 'Phone Number', 'SSN'])['count'].sum().reset_index(name='count')
stacked_roll_1["count"].describe()

count    9.498655e+07
mean     1.117462e+00
std      4.401064e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.898300e+04
Name: count, dtype: float64

In [ ]:
freq("count", df = stacked_roll_1)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,87892305.0,9.253131e-01,87892305.0,0.925313
2,6163960.0,6.489298e-02,94056265.0,0.990206
3,519241.0,5.466469e-03,94575506.0,0.995673
4,173453.0,1.826080e-03,94748959.0,0.997499
5,61449.0,6.469232e-04,94810408.0,0.998146
...,...,...,...,...
4829,1.0,1.052781e-08,94986548.0,1.000000
5152,1.0,1.052781e-08,94986549.0,1.000000
8221,1.0,1.052781e-08,94986550.0,1.000000


In [10]:
stacked_roll_1[stacked_roll_1["count"] == 18983]

,date,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN,count
17978875,2021-07-23,ISTCMSTEST322,,915 E BRETT ST,,INGLEWOOD,CA,,US,000000000000,,18983


In [12]:
stacked_roll_1[stacked_roll_1["count"] == 16930]

,date,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN,count
18056000,2021-07-26,ISTCMSTEST322,,915 E BRETT ST,,INGLEWOOD,CA,,US,000000000000,,16930


In [16]:
stacked_roll_1[stacked_roll_1["count"] == 8221]

,date,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN,count
50828121,2021-12-17,BUSINESS CARD,,,,,,,,,.,8221


In [9]:
fmt = make_format(cuts = [-np.inf, -1, 0, 1, 2, 3, 4, 5, 10, 50, 100, np.inf])
freq("count", df = stacked_roll_1, format = fmt, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,87892305.0,0.925313,87892305.0,0.925313
2,6163960.0,0.064893,94056265.0,0.990206
3,519241.0,0.005466,94575506.0,0.995673
4,173453.0,0.001826,94748959.0,0.997499
5,61449.0,0.000647,94810408.0,0.998146
6-10,101703.0,0.001071,94912111.0,0.999216
11-50,70701.0,0.000744,94982812.0,0.999961
51-100,1229.0,0.000013,94984041.0,0.999974
101+,2511.0,0.000026,94986552.0,1.000000


#### round 3

In [20]:
## by week
stacked["date"] = stacked["Date/Time"].str.slice(0,10)
stacked["date_fmt"] = pd.to_datetime(stacked["date"]) - pd.to_timedelta(7, unit='d')
stacked.tail()

,SBFE Credit Inquiry ID,Record Type,Version Number,Date/Time,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN,count,date,date_fmt
106154102,AAA000000119903161,CI,0.1,2022-06-30T23:57:21,DEMETRAS DOUGLAS,,119 OAK ST,,GRANITEVILLE,SC,298293029,,,862400814,1,2022-06-30,2022-06-23
106154103,AAA000000119903162,CI,0.1,2022-06-30T23:57:33,DOMINON AND ROYALTY,,4860 KINGS CT,,RICHTON PARK,IL,604711605,,,874103432,1,2022-06-30,2022-06-23
106154104,AAA000000119903163,CI,0.1,2022-06-30T23:57:52,STEWART PATTEN CO LLC,,1511 HOLCOMBS POND CT,,ALPHARETTA,GA,300222870,,,941611234,1,2022-06-30,2022-06-23
106154105,AAA000000119903164,CI,0.1,2022-06-30T23:58:30,SOUL OF WIT PRODUCTIONS,,1735 LONGVIEW WAY NW,,SALEM,OR,973044434,,,544519595,1,2022-06-30,2022-06-23
106154106,AAA000000119903165,CI,0.1,2022-06-30T23:58:36,"TRANSPORT ME, INC.",,13145 YERBA ST,,N HOLLYWOOD,CA,916051829,,,261602098,1,2022-06-30,2022-06-23


In [21]:
stacked_roll_1 =stacked.groupby(by = [pd.Grouper(key='date_fmt', freq='W-MON'), 
                                      'Business Name', 'DBA', 'Address1', 'Address2', 'City', 'State', 'Zip4',
                                      'Contry Code', 'Phone Number', 'SSN'])['count'].sum().reset_index(name='count')
freq("count", df = stacked_roll_1)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,75848024.0,8.620251e-01,75848024.0,0.862025
2,9726585.0,1.105442e-01,85574609.0,0.972569
3,1887875.0,2.145601e-02,87462484.0,0.994025
4,220339.0,2.504189e-03,87682823.0,0.996530
5,80523.0,9.151570e-04,87763346.0,0.997445
...,...,...,...,...
11706,2.0,2.273033e-08,87988176.0,1.000000
11745,1.0,1.136516e-08,87988177.0,1.000000
11757,1.0,1.136516e-08,87988178.0,1.000000


In [22]:
stacked_roll_1[stacked_roll_1["count"] == 40742]

,date_fmt,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN,count
13335914,2021-07-19,ISTCMSTEST322,,915 E BRETT ST,,INGLEWOOD,CA,,US,000000000000,,40742


In [23]:
fmt = make_format(cuts = [-np.inf, -1, 0, 1, 2, 3, 4, 5, 10, 50, 100, np.inf])
freq("count", df = stacked_roll_1, format = fmt, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,75848024.0,0.862025,75848024.0,0.862025
2,9726585.0,0.110544,85574609.0,0.972569
3,1887875.0,0.021456,87462484.0,0.994025
4,220339.0,0.002504,87682823.0,0.996530
5,80523.0,0.000915,87763346.0,0.997445
6-10,136313.0,0.001549,87899659.0,0.998994
11-50,86360.0,0.000981,87986019.0,0.999975
51-100,1180.0,0.000013,87987199.0,0.999989
101+,981.0,0.000011,87988180.0,1.000000


#### Round 4

In [24]:
## by week
stacked["date"] = stacked["Date/Time"].str.slice(0,7)
stacked_roll_1 =stacked.groupby(by = ['date', 
                                      'Business Name', 'DBA', 'Address1', 'Address2', 'City', 'State', 'Zip4',
                                      'Contry Code', 'Phone Number', 'SSN'])['count'].sum().reset_index(name='count')
freq("count", df = stacked_roll_1)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,51731402.0,7.148889e-01,51731402.0,0.714889
2,12954096.0,1.790158e-01,64685498.0,0.893905
3,5532446.0,7.645423e-02,70217944.0,0.970359
4,1777264.0,2.456045e-02,71995208.0,0.994919
5,111836.0,1.545489e-03,72107044.0,0.996465
...,...,...,...,...
49543,1.0,1.381924e-08,72362850.0,1.000000
49727,1.0,1.381924e-08,72362851.0,1.000000
50735,1.0,1.381924e-08,72362852.0,1.000000


In [26]:
stacked_roll_1[stacked_roll_1["count"] == 51834]

,date,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN,count
55196289,2022-03,"GORMAN MANUFACTURING COMPANY, INC.",,492 KOLLER ST,,SAN FRANCISCO,CA,94110,US,6505550000,,51834


In [28]:
stacked_roll_1[stacked_roll_1["count"] == 50735]

,date,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN,count
66538280,2022-05,"GORMAN MANUFACTURING COMPANY, INC.",,492 KOLLER ST,,SAN FRANCISCO,CA,94110,US,6505550000,,50735


In [25]:
freq("count", df = stacked_roll_1, format = fmt, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,51731402.0,0.714889,51731402.0,0.714889
2,12954096.0,0.179016,64685498.0,0.893905
3,5532446.0,0.076454,70217944.0,0.970359
4,1777264.0,0.024560,71995208.0,0.994919
5,111836.0,0.001545,72107044.0,0.996465
6-10,160072.0,0.002212,72267116.0,0.998677
11-50,93347.0,0.001290,72360463.0,0.999967
51-100,1576.0,0.000022,72362039.0,0.999989
101+,815.0,0.000011,72362854.0,1.000000


#### Round5

In [29]:
stacked_roll_1 =stacked.groupby(by = [
       'Business Name', 'DBA', 'Address1', 'Address2', 'City', 'State', 'Zip4',
       'Contry Code', 'Phone Number', 'SSN'])['count'].sum().reset_index(name='count')
freq("count", df = stacked_roll_1)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,9812212.0,4.466622e-01,9812212.0,0.446662
2,4689777.0,2.134836e-01,14501989.0,0.660146
3,899667.0,4.095379e-02,15401656.0,0.701100
4,758014.0,3.450560e-02,16159670.0,0.735605
5,321981.0,1.465691e-02,16481651.0,0.750262
...,...,...,...,...
45946,1.0,4.552105e-08,21967853.0,1.000000
54648,1.0,4.552105e-08,21967854.0,1.000000
66412,1.0,4.552105e-08,21967855.0,1.000000


In [30]:
stacked_roll_1[stacked_roll_1["count"] == 554794]

,Business Name,DBA,Address1,Address2,City,State,Zip4,Contry Code,Phone Number,SSN,count
8071563,"GORMAN MANUFACTURING COMPANY, INC.",,492 KOLLER ST,,SAN FRANCISCO,CA,94110,US,6505550000,,554794


In [31]:
freq("count", df = stacked_roll_1, format = fmt, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,9812212.0,0.446662,9812212.0,0.446662
2,4689777.0,0.213484,14501989.0,0.660146
3,899667.0,0.040954,15401656.0,0.701100
4,758014.0,0.034506,16159670.0,0.735605
5,321981.0,0.014657,16481651.0,0.750262
6-10,1381803.0,0.062901,17863454.0,0.813163
11-50,4101423.0,0.186701,21964877.0,0.999864
51-100,1975.0,0.000090,21966852.0,0.999954
101+,1005.0,0.000046,21967857.0,1.000000


In [19]:
freq(stacked["Contry Code"])

,Count,Pct,Cuml Count,Cuml Pct
Contry Code,,,,
US,90921068.0,0.856501,90921068.0,0.856501
Missing,15233039.0,0.143499,106154107.0,1.000000


In [ ]:
## 
stacked["Business Name"] != "#"

## LN Reject Data

In [4]:
header = ['file_name', 'no_file_issues', 'search_information_missing', 'duplicate_record', 'sbfe_contribution_id_missing', 'date_time_missing', 
'platform_type_missing', 'hit_no_hit_indicator_missing', 'too_many_delimeters', 'cv_id', 'cv_customer_id', 'cv_customer_name', 'sbfe_member_id', 
'datetime', 'platform_type', 'hit_no_hit_indicator', 'base_record_filler_1', 'base_record_filler_2', 'business_name', 'dba_name', 'address_1', 
'address_2', 'city', 'state', 'zip', 'postal_code', 'country_code', 'phone_number', 'business_tax_id', 'borrower_filler_1', 'borrower_filler_2', 
'rep_first_name', 'rep_last_name', 'rep_generation_code', 'rep_address_1', 'rep_address_2', 'rep_city', 'rep_state', 'rep_zip', 'rep_postal_code', 
'rep_country_code', 'rep_phone_number', 'rep_ssn', 'rep_date_of_birth', 'rep_drivers_license_number', 'rep_drivers_license_state', 'rep_drivers_license_expiration_date', 
'guarantor_filler_1', 'guarantor_filler_2', 'jobtx_id', 'sequencenumber', "extra1","extra2","extra3"]

In [11]:
ln_reject.loc[0]

file_name                              batch_mCVLNB_i2095714_d20221010184147_LNRS_INQ...
no_file_issues                                                                         0
search_information_missing                                                             0
duplicate_record                                                                       0
sbfe_contribution_id_missing                                                           1
date_time_missing                                                                      0
platform_type_missing                                                                  0
hit_no_hit_indicator_missing                                                           1
too_many_delimeters                                                                    0
cv_id                                                                               CVLN
cv_customer_id                                                                   1469887
cv_customer_name     

In [13]:
ln_reject = pd.read_adls(path + path_ln_reject, reader = pd.read_csv, sep = "\t", names=header, skiprows = [0], dtype = str)
print(ln_reject.shape)

(11188041, 54)


In [14]:
ln_reject.extra1.notnull().sum()

5

In [15]:
ln_reject_1 = ln_reject[ln_reject["extra1"].isnull()].copy()
temp = ln_reject[ln_reject["extra1"].notnull()].copy()

In [16]:
temp = temp.drop(['city'], axis=1)
temp["extra4"] = ""
temp.columns = ln_reject_1.columns

In [17]:
ln_reject = pd.concat([ln_reject_1, temp], ignore_index = True)
ln_reject = ln_reject.drop(['extra1', 'extra2', "extra3"], axis=1)
ln_reject.shape

(11188041, 51)

In [18]:
ln_reject.to_adls(path + "ln_reject.parquet", format = '.parquet', overwrite = True)

In [5]:
ln_reject = pd.read_adls(path + "ln_reject.parquet", reader = pd.read_parquet)
print(ln_reject.shape)

(11188041, 51)


In [6]:
from dsgtools.reporting import TableWriter

In [21]:
wb = TableWriter(filename = "./_temp/test.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
wb.write_table(
    pd.DataFrame(ln_reject.isnull().sum()),
    sheetname="state",
)
wb.close()

In [7]:
result = dict()
result["f1"] = freq('no_file_issues', df = ln_reject)
result["f2"] = freq('search_information_missing', df = ln_reject)
result["f3"] = freq('duplicate_record', df = ln_reject)
result["f4"] = freq('sbfe_contribution_id_missing', df = ln_reject)
result["f5"] = freq('date_time_missing', df = ln_reject)
result["f6"] = freq('platform_type_missing', df = ln_reject)
result["f7"] = freq('hit_no_hit_indicator_missing', df = ln_reject)
result["f8"] = freq('too_many_delimeters', df = ln_reject)
result["f9"] = freq('cv_customer_name', df = ln_reject)
result["f10"] = freq('platform_type', df = ln_reject)
result["f11"] = freq('too_many_delimeters', df = ln_reject)
result["f12"] = freq('jobtx_id', df = ln_reject)


wb = TableWriter(filename = "./_temp/test.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="ln_reject",
    )
wb.close()

In [8]:
ln_reject["date"] = ln_reject.datetime.str.slice(0,7)
freq(ln_reject["date"])

,Count,Pct,Cuml Count,Cuml Pct
date,,,,
2021-07,345187.0,0.030853,345187.0,0.030853
2021-08,100148.0,0.008951,445335.0,0.039805
2021-09,2180106.0,0.194860,2625441.0,0.234665
2021-10,324115.0,0.028970,2949556.0,0.263635
2021-11,101486.0,0.009071,3051042.0,0.272706
2021-12,2241685.0,0.200364,5292727.0,0.473070
2022-01,197432.0,0.017647,5490159.0,0.490717
2022-02,107671.0,0.009624,5597830.0,0.500340
2022-03,2398272.0,0.214360,7996102.0,0.714701


Bad pipe message: %s [b'\xb8\xcc\xc4\xfeJ+J\x8e\x02\xa6,\xf5CL\xdb\xdf\x85\xe0 \n\xb0\n\x81\xda\x982\x9a\xb9\xeb\x9e\x96\x9f\x97\x95b_m\x89\xea\x9c\n\tK\xc8L\xa4?\x8co\xd3a\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00']
Bad pipe message: %s [b'\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07']
Bad pipe message: %s [b'\x08\t\x08\n\x08\x0b\x08']
Bad pipe message: %s [b"7@\xd5\xbc1a\xa5\xf0OP\xe5\x92\xeb\x00\xdb[4X\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96

In [11]:
ln_reject.sbfe_member_id.notnull().sum()

0